### RUL Target Computation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [13]:
# 1. Load rolling windows sequence (NumPy array)
sequences = np.load('rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequences.shape}")

Loaded rolling window sequences with shape: (10, 5, 3)


In [12]:
# Create a dummy numpy array and save it as a .npy file
dummy_sequences = np.random.rand(10, 5, 3) # Example shape, adjust as needed
np.save('rolling_window_sequences.npy', dummy_sequences)
print("Created a dummy 'rolling_window_sequences.npy' file.")

Created a dummy 'rolling_window_sequences.npy' file.


In [23]:
# 2. Load sequence metadata (engine_id, cycle)
metadata_df = pd.read_csv('sequence_metadata.csv')
print(f"Loaded sequence metadata with shape: {metadata_df.shape}")
display(metadata_df.head())

Loaded sequence metadata with shape: (10, 2)


,engine_id,cycle
0,1,1
1,1,10
2,2,1
3,2,12
4,3,1


In [22]:
# Create a dummy sequence_metadata.csv file
import pandas as pd
import numpy as np

# Assuming your sequences have a shape like (num_sequences, window_size, num_features)
# And each sequence corresponds to a unique engine_id and cycle
num_sequences = 10 # Matches the dummy sequences created earlier
dummy_metadata = pd.DataFrame({
    'engine_id': np.repeat(np.arange(1, num_sequences // 2 + 1), 2), # Example: 5 engines, 2 sequences each
    'cycle': np.tile(np.arange(1, 3), num_sequences // 2) # Example cycles
})

# Introduce some variation in max cycles per engine for testing RUL calculation
dummy_metadata.loc[dummy_metadata['engine_id'] == 1, 'cycle'] = [1, 10]
dummy_metadata.loc[dummy_metadata['engine_id'] == 2, 'cycle'] = [1, 12]
dummy_metadata.loc[dummy_metadata['engine_id'] == 3, 'cycle'] = [1, 8]
dummy_metadata.loc[dummy_metadata['engine_id'] == 4, 'cycle'] = [1, 15]
dummy_metadata.loc[dummy_metadata['engine_id'] == 5, 'cycle'] = [1, 11]


dummy_metadata.to_csv('sequence_metadata.csv', index=False)
print("Created a dummy 'sequence_metadata.csv' file.")
display(dummy_metadata.head())

Created a dummy 'sequence_metadata.csv' file.


,engine_id,cycle
0,1,1
1,1,10
2,2,1
3,2,12
4,3,1


In [24]:
# 3. Calculate Remaining Useful Life (RUL) per sequence

# Find max cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,10
1,2,12
2,3,8
3,4,15
4,5,11


In [25]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()


,engine_id,cycle,max_cycle
0,1,1,10
1,1,10,10
2,2,1,12
3,2,12,12
4,3,1,8


In [26]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [ ]:
metadata_df.head()

,engine_id,cycle,max_cycle,RUL
0,1,1,10,9
1,1,10,10,0
2,2,1,12,11
3,2,12,12,0
4,3,1,8,7


In [27]:
# Optional: Cap RUL values
# rul_cap = 130
# metadata_df['RUL'] = metadata_df['RUL'].clip(upper=rul_cap)

# Drop max_cycle column if not needed further
metadata_df.drop(columns=['max_cycle'], inplace=True)

metadata_df.head()

,engine_id,cycle,RUL
0,1,1,9
1,1,10,0
2,2,1,11
3,2,12,0
4,3,1,7


In [28]:
# 4. Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3]  # first 3 engines
for engine in engines_to_check:
    sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
    print(f"\nEngine {engine} - Cycles and RULs:")
    print(sample.head(10))
    print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle  RUL
0      1    9
1     10    0
   cycle  RUL
0      1    9
1     10    0

Engine 2 - Cycles and RULs:
   cycle  RUL
2      1   11
3     12    0
   cycle  RUL
2      1   11
3     12    0

Engine 3 - Cycles and RULs:
   cycle  RUL
4      1    7
5      8    0
   cycle  RUL
4      1    7
5      8    0


In [29]:
# 5. Save updated metadata with RUL target
metadata_df.to_csv('sequence_metadata_with_RUL.csv', index=False)
print("Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'")

Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'


**Process Steps:**  
- Loaded rolling window feature sequences from 'rolling_windows_feature.npy'.  
- Loaded sequence metadata from 'sequence_metadata.csv' including engine and cycle identifiers.  
- Computed Remaining Useful Life (RUL) for each sequence as the difference between
  maximum cycle for the engine and the current cycle.  
- Optionally applied RUL capping at 130 to stabilize regression targets.  
- Saved updated metadata including RUL targets for downstream model training.

**Outputs:**  
- Rolling window sequences (NumPy array) ready for modeling.  
- Updated sequence metadata with assigned RUL targets.  

**Next Steps:**  
- Use these datasets for training and evaluating LSTM/GRU predictive models for RUL.  

---

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,GRU

In [38]:
x= np.load('rolling_window_sequences.npy')
y= pd.read_csv('sequence_metadata_with_RUL.csv')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [41]:
model=Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train['RUL'], epochs=10, batch_size=32, validation_data=(X_test, y_test['RUL']))

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 56.6066 - val_loss: 50.5428
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 56.3098 - val_loss: 50.2530
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 56.0180 - val_loss: 49.9671
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 55.7302 - val_loss: 49.6837
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 55.4453 - val_loss: 49.4015
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 55.1617 - val_loss: 49.1194
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 54.8782 - val_loss: 48.8363
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 54.5933 - val_loss: 48.5509
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 54.3057 - val_loss: 48.2622
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 54.0141 - val_loss: 47.9687


In [42]:
from sklearn.metrics import r2_score
import numpy as np
y_pred=model.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test['RUL'], y_pred))
r2=r2_score(y_test['RUL'], y_pred)
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
RMSE: 6.925942906936678
R2: -0.9187474250793457


In [45]:
model=Sequential()
model.add(GRU(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train['RUL'], epochs=10, batch_size=32, validation_data=(X_test, y_test['RUL']))


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 54.9494 - val_loss: 48.3657
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 54.4972 - val_loss: 47.8976
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 54.0516 - val_loss: 47.4349
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - loss: 53.6111 - val_loss: 46.9758
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 53.1739 - val_loss: 46.5186
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 52.7379 - val_loss: 46.0616
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 52.3015 - val_loss: 45.6033
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 51.8628 - val_loss: 45.1421
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 51.4203 - val_loss: 44.6765
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 50.9723 - val_loss: 44.2049


In [48]:
y_pred_gru = model.predict(X_test)
rmse_gru = np.sqrt(mean_squared_error(y_test['RUL'], y_pred_gru))
r2_gru = r2_score(y_test['RUL'], y_pred_gru)

print(f"GRU Model RMSE: {rmse_gru}")
print(f"GRU Model R2: {r2_gru}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
GRU Model RMSE: 6.64867934113243
GRU Model R2: -0.7681975364685059
